In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.io import loadmat
from scipy.signal import ellip, lfilter, filtfilt, find_peaks, butter, sosfiltfilt,sosfilt

In [7]:
%matplotlib inline

In [8]:
feats_s1_train = np.load('all_feats_s1_train_fx.npy')
feats_s2_train = np.load('all_feats_s2_train_fx.npy')
feats_s3_train = np.load('all_feats_s3_train_fx.npy')

feats_s1_test = np.load('all_feats_s1_test_fx.npy')
feats_s2_test = np.load('all_feats_s2_test_fx.npy')
feats_s3_test = np.load('all_feats_s3_test_fx.npy')

# Normalizing the features

This script will normalize the features based on the training set

In [9]:
def normalize_features(all_feats_train, all_feats_test, num_features):
    # Input should be (num_windows x (channels x features))
    # Num features is the number of unique features that were extracted
    all_feats_train_norm = np.copy(all_feats_train)
    all_feats_test_norm = np.copy(all_feats_test)
    
    feats_avg = []
    feats_std = []
    
    for n in range(num_features):
        feats_idx_train = np.arange(n,len(all_feats_train.transpose()),num_features)
        
        feat_data_train = all_feats_train[:][:,feats_idx_train]
        
        feat_means_train = np.mean(feat_data_train,axis=0)
        feat_stds_train = np.std(feat_data_train,axis=0)
        
        all_feats_train_norm[:][:,feats_idx_train] = (feat_data_train - feat_means_train)/feat_stds_train
        
        # Note that we must use the same mean and std. dev from the TRAINING set
        #     because regression models are sensitive to value domain as they are
        #     scale-variant.
        feats_idx_test = np.arange(n,len(all_feats_test.transpose()),num_features)
        feat_data_test = all_feats_test[:][:,feats_idx_test]
        all_feats_test_norm[:][:,feats_idx_test] = (feat_data_test - feat_means_train)/feat_stds_train
        
        # Sanity checking plot, comment out if you don't want plots
#         if n == 0:
#             plt.figure()
#             plt.plot(feat_data_train.transpose()[0])
#             plt.figure()
#             plt.plot(all_feats_train_norm[:][:,feats_idx_train].transpose()[0])
            
#             plt.figure()
#             plt.plot(feat_data_test.transpose()[0])
#             plt.figure()
#             plt.plot(all_feats_test_norm[:][:,feats_idx_test].transpose()[0])

    return all_feats_train_norm, all_feats_test_norm
    

In [10]:
# NOTE: SPECIFY THIS MANUALLY, 
#       it's difficult to backwards engineer the feature number after flattening
num_features = 7 # !!!!!!!!!!!!

feats_s1_train_norm, feats_s1_test_norm = normalize_features(feats_s1_train, feats_s1_test, num_features)
feats_s2_train_norm, feats_s2_test_norm = normalize_features(feats_s2_train, feats_s2_test, num_features)
feats_s3_train_norm, feats_s3_test_norm = normalize_features(feats_s3_train, feats_s3_test, num_features)

<ipython-input-9-37efb986875c>:18: RuntimeWarning: invalid value encountered in true_divide
  all_feats_train_norm[:][:,feats_idx_train] = (feat_data_train - feat_means_train)/feat_stds_train
<ipython-input-9-37efb986875c>:25: RuntimeWarning: invalid value encountered in true_divide
  all_feats_test_norm[:][:,feats_idx_test] = (feat_data_test - feat_means_train)/feat_stds_train


# Saving to files

Specify your file names and directory as necessary.

In [11]:
# Save the training features
file = open("all_feats_s1_norm_train_fx.npy","wb")
np.save(file, feats_s1_train_norm)
file.close()

file = open("all_feats_s2_norm_train_fx.npy","wb")
np.save(file, feats_s2_train_norm)
file.close()

file = open("all_feats_s3_norm_train_fx.npy","wb")
np.save(file, feats_s3_train_norm)
file.close()

# Save the testing features
file = open("all_feats_s1_norm_test_fx.npy","wb")
np.save(file, feats_s1_test_norm)
file.close()

file = open("all_feats_s2_norm_test_fx.npy","wb")
np.save(file, feats_s2_test_norm)
file.close()

file = open("all_feats_s3_norm_test_fx.npy","wb")
np.save(file, feats_s3_test_norm)
file.close()